In [3]:
import argparse
import collections
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer
from utils import prepare_device
from experiments import get_experiment_config

In [13]:
args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default=None, type=str,
                      help='config file path (default: None)')
args.add_argument('-r', '--resume', default=None, type=str,
                      help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default=None, type=str,
                  help='indices of GPUs to enable (default: all)')
# args
# args.c = "config.json"
# args.exp = 0
args = args.parse_args(['-c','config.json'])
print(args)
config = ConfigParser.from_args(args)
config = get_experiment_config(config, config['id'])

Namespace(config='config.json', resume=None, device=None)


AttributeError: 'Namespace' object has no attribute 'parse_args'